![header](Copernicus_Marine/demo_cmems_diags/figures/logos.jpg)

# Copernicus Marine Sea Level Demo

<div style="text-align: right"><i> 2022-08-03 SEALEVEL_DEMO </i></div>

***

# How to quantify errors in Copernicus Sea Level products ?


***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***

## 0. Environment Setup

At first, let's setup the notebook with all the necessary tools available from the Jupyter Notebook Ecosystem.

#### Importing modules
* Unlike MATLAB, Python libraries need to be `imported` before they can be used
* Imported libraries often have a short name ("namespace")  (not mandatory, though; you'll most often find xarray shortened as xr and numpy as np. For lisibility we choose not to, except for matplotlib.pyplot and cartopy.crs, a bit long)
* Portions of libraries can be imported

In [ ]:
import warnings
import os
import sys

import numpy as np
import numpy
import xarray as xr
import getpass
import logging

import hvplot.xarray
import pyinterp
import pyinterp.backends.xarray

import pandas
import datetime
from datetime import timedelta

%matplotlib inline
warnings.simplefilter("ignore")

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(module)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [ ]:
sys.path.append('..')

In [ ]:
from src.mod_io import *
from src.mod_plot import *
from src.mod_eddy import *
from src.mod_interp import *
from src.mod_scores import *
from src.mod_spectral import *

<hr>

### Create an account on Copernicus Marine Service: https://resources.marine.copernicus.eu/registration-form

### Login

In [ ]:
USERNAME = 'mballarotta1'
PASSWORD = getpass.getpass('Enter your password: ')

## 1. Copernicus Products

We will focus on the following product (available on the [Copernicus Catalogue](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&task=results)):
- [SEALEVEL_GLO_PHY_L4_MY_008_047](https://resources.marine.copernicus.eu/product-detail/SEALEVEL_GLO_PHY_L4_MY_008_047/INFORMATION)
- [SEALEVEL_GLO_PHY_L3_MY_008_062](https://resources.marine.copernicus.eu/product-detail/SEALEVEL_GLO_PHY_L3_MY_008_062/INFORMATION)

In [ ]:
time_min = '2017-01-01'
time_max = '2017-12-31'

### 1.1 Alongtrack sea level data (SARAL/Altika)

In [ ]:
# ds_alg = get_cmems_duacs_alongtrack('alg', time_min , time_max, USERNAME, PASSWORD)
# ds_alg

In [ ]:
# TODO: add ftp solution

In [ ]:
ds_alg = xr.open_mfdataset('/data/PVA/v2021/Interne/ftp/CMEMS/global/dt-along-track/alg/2017/dt_global_alg_phy_l3_2017*.nc', combine='nested', concat_dim='time')
ds_alg = ds_alg.where((ds_alg.time >= np.datetime64(time_min)) & (ds_alg.time <=  np.datetime64(time_max)), drop=True)
ds_alg

### 1.2 Optimal interpolation sea level maps

In [ ]:
DUACS_L4_DT_DATASET_ID = 'cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D'

In [ ]:
data_store_duacs_dt2021 = copernicusmarine_datastore(DUACS_L4_DT_DATASET_ID, USERNAME, PASSWORD)

In [ ]:
ds_duacs_dt2021 = xr.open_dataset(data_store_duacs_dt2021)

In [ ]:
ds_duacs_dt2021_selection = ds_duacs_dt2021.sel(time=slice(time_min, time_max))
ds_duacs_dt2021_selection

## 2. Statistical analysis: comparing sea level maps and along-track data

### 2.1 Interpolate sea level anomaly map onto along-track position

In [ ]:
ds_interp = run_interpolation(ds_duacs_dt2021_selection, ds_alg)
ds_interp

### 2.2 Compute scores between maps and along-track

In [ ]:
compute_stats_map(ds_interp, 'stats_map.nc')

In [ ]:
compute_stats_timeseries(ds_interp, 'stats_timeseries.nc')

In [ ]:
compute_psd_scores(ds_interp, 'psd_score.nc')

### 2.3 Visualize scores

In [ ]:
plot_map_scores('stats_map.nc')

In [ ]:
plot_timeseries_scores('stats_timeseries.nc')

In [ ]:
plot_psd_scores('psd_score.nc')